In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

import torch

torch.cuda.device_count()

AttributeError: module 'torch' has no attribute 'cuda'

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../../smoothing/code')
sys.path.append('../../../SupContrast/')

from datasets import get_dataset

device = 'cuda'
dataset = "cifar10"

test_dataset = get_dataset(dataset, "test")
train_dataset = get_dataset(dataset, "train")

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False, num_workers=16)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Files already downloaded and verified
Files already downloaded and verified


In [3]:
from datasets import get_normalize_layer
import torch
from networks.resnet_big import SupConResNet
from core import Smooth

base_path = "/home/mehrshad_mirmohammadi/Project/SupContrast/save/SupCon/cifar10_models/SupCon_cifar10_resnet50_lr_0.5_decay_0.0001_bsz_1024_temp_0.1_trial_0_cosine_warm_sigma_0.25/"
weights_file = f"{base_path}ckpt_epoch_1000.pth"

sigma = float(base_path.split("_")[-1][:-1])
print(sigma)


class Classifier(torch.nn.Module):
    def __init__(
        self,
        encoder_weights,
        predictor,
        dataset="cifar10",
    ):
        super(Classifier, self).__init__()
        self.encoder = SupConResNet(name="resnet50", head="mlp", feat_dim=128)

        state_dict = torch.load(encoder_weights)

        renamed_state_dict = {}
        for k, v in state_dict["model"].items():
            renamed_state_dict[k.replace("module.", "")] = v

        self.encoder.load_state_dict(renamed_state_dict)
        self.encoder = self.encoder.encoder.to(device)

        self.set_fine_tune(False)

        self.predictor = predictor

        self.normalizer = get_normalize_layer(dataset)
        self.normalizer.means = self.normalizer.means.to(device)
        self.normalizer.sds = self.normalizer.sds.to(device)
        self.normalizer.eval()


    def set_fine_tune(self, fine_tune):
        self.fine_tune = fine_tune

        for param in self.encoder.parameters():
            param.requires_grad = fine_tune
        self.encoder.train(fine_tune)

    def forward(self, x):
        x = self.normalizer(x)

        if self.fine_tune:
            x = self.encoder(x)
        else:
            with torch.no_grad():
                x = self.encoder(x)

        x = self.predictor(x)

        return x


predictor = torch.nn.Sequential(
    torch.nn.BatchNorm1d(2048),
    torch.nn.Linear(2048, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10),
).to(device)

classifier = Classifier(weights_file, predictor).to(device)

print(
    f"Number of parameters in the Encoder: {sum([p.numel() for p in classifier.encoder.parameters()]) / 1000_000}"
)
print(
    f"Number of parameters in the Predictor: {sum([p.numel() for p in classifier.predictor.parameters()]) / 1000_000}"
)
print(
    f"Number of parameters in the Classifier: {sum([p.numel() for p in classifier.parameters()]) / 1000_000}"
)
print(
    f"Number of trainable parameters in the Classifier: {sum([p.numel() for p in classifier.parameters() if p.requires_grad]) / 1000_000}"
)

from torchinfo import summary

summary(classifier, input_size=(1, 3, 32, 32))


ModuleNotFoundError: No module named 'datasets'

In [16]:
from tqdm import tqdm

# train the classifier
import torch.optim as optim
import torch.nn.functional as F

optimizer = optim.Adam(predictor.parameters(), lr=0.02)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15, eta_min=0.01)

for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    running_accuracy = 0.0
    
    classifier.train()
    tq = tqdm(enumerate(train_loader, 0), total=len(train_loader))
    for i, batch in tq:

        (inputs, labels) = batch
        inputs = inputs.to(device)

        noise = torch.rand_like(inputs, device=device) * sigma
        inputs  += noise

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = classifier(inputs)
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        # calc accuracy
        _, predicted = torch.max(outputs.cpu().data, 1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / labels.size(0)

        running_loss += loss.item()
        running_accuracy += accuracy

        # print statistics
        tq.set_description(f'Epoch {epoch} - Running Loss: {running_loss / (i + 1):.4f} - Running Accuracy: {running_accuracy / (i + 1):.4f}')
    
    scheduler.step()

    classifier.eval()

    # accuracy on test set
    correct = 0
    total = 0

    with torch.no_grad():
        
        for i, batch in enumerate(test_loader, 0):

            (inputs, labels) = batch
            inputs = inputs.to(device)

            outputs = classifier(inputs).cpu()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).cpu().sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')

    # accuracy on test set
    correct = 0
    total = 0

    with torch.no_grad():
        
        for i, batch in enumerate(test_loader, 0):

            (inputs, labels) = batch
            inputs = inputs.to(device)

            noise = torch.rand_like(inputs, device=device) * sigma
            inputs  += noise

            outputs = classifier(inputs).cpu()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).cpu().sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}% | Using noise with sigma={sigma}')

100%|██████████| 1000/1000 [44:24<00:00,  2.66s/it]


Epoch 0 - Running Loss: 0.5570 - Running Accuracy: 0.8262: 100%|██████████| 98/98 [00:07<00:00, 13.42it/s]


Accuracy of the network on the 10000 test images: 93.51%
Accuracy of the network on the 10000 test images: 82.56% | Using noise with sigma=0.25


Epoch 1 - Running Loss: 0.4719 - Running Accuracy: 0.8443: 100%|██████████| 98/98 [00:07<00:00, 13.75it/s]


Accuracy of the network on the 10000 test images: 93.52%
Accuracy of the network on the 10000 test images: 82.59% | Using noise with sigma=0.25


...

Accuracy of the network on the 10000 test images: 93.63%
Accuracy of the network on the 10000 test images: 83.48% | Using noise with sigma=0.25


Epoch 13 - Running Loss: 0.4133 - Running Accuracy: 0.8610: 100%|██████████| 98/98 [00:07<00:00, 13.59it/s]


Accuracy of the network on the 10000 test images: 93.65%
Accuracy of the network on the 10000 test images: 82.9% | Using noise with sigma=0.25


Epoch 14 - Running Loss: 0.4117 - Running Accuracy: 0.8590: 100%|██████████| 98/98 [00:07<00:00, 13.50it/s]


Accuracy of the network on the 10000 test images: 93.46%
Accuracy of the network on the 10000 test images: 83.36% | Using noise with sigma=0.25


In [17]:
# # Save the model checkpoint
# torch.save(classifier.state_dict(), f'{super_path}one_layer.pth')

In [18]:
from time import time
import datetime
import os

# sigma = 0.5
iter_skip = 10
iter_max = 50000
N0 = 100
N = 8_000
alpha = 0.01
batch_size = 8_000

smoothed_classifier = Smooth(classifier, 10, sigma)

# prepare output file

# set exp_name to the architecture of the Predictor

arch_name = ''

for layer in predictor.children():
    if isinstance(layer, torch.nn.Linear):
        arch_name += f'_{layer.out_features}'
    elif isinstance(layer, torch.nn.BatchNorm1d):
        arch_name += '_bn'

exp_name = "reduced{}".format(arch_name)

out_file = f"{base_path}RS/{exp_name}_testsigma_{sigma}_time_{datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}.txt"
os.makedirs(os.path.dirname(out_file), exist_ok=True)

print(out_file)

f = open(out_file, "w")
print("idx\tlabel\tpredict\tradius\tcorrect\ttime", file=f, flush=True)

from tqdm import tqdm

tq = tqdm(total=len(test_dataset) // iter_skip)

for i, batched_sample in enumerate(test_dataset):
    # only certify every args.skip examples, and stop after args.max examples
    if i % iter_skip != 0:
        continue
    if i == iter_max:
        break

    (x, label) = test_dataset[i]

    before_time = time()
    # certify the prediction of g around x
    x = x.cuda()
    prediction, radius = smoothed_classifier.certify(x, N0, N, alpha, batch_size)
    after_time = time()
    correct = int(prediction == label)

    time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
    print(
        "{}\t{}\t{}\t{:.3}\t{}\t{}".format(
            i, label, prediction, radius, correct, time_elapsed
        ),
        file=f,
        flush=True,
    )
    # print(
    #     "{}\t{}\t{}\t{:.3}\t{}\t{}".format(
    #         i, label, prediction, radius, correct, time_elapsed
    #     )
    # )
    tq.update(1)


f.close()

/home/mehrshad_mirmohammadi/Project/SupContrast/save/SupCon/cifar10_models/SupCon_cifar10_resnet50_lr_0.5_decay_0.0001_bsz_1024_temp_0.1_trial_0_cosine_warm_sigma_0.25/RS/reduced_10_testsigma_0.25_time_2023-08-03_13:23:20.txt


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [15:07<00:00,  1.10it/s]